In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sls

In [13]:
candidates = pd.read_json("election_datas/CumhurbaskaniAdayListesi.json")
candidates["Adı Soyadı"] = candidates["Adı"]+" "+candidates["Soyadı"]
new_order = ["Adı Soyadı","Üniversite/Yüksekokul","Doğum Tarihi","Aday Sıra No"]
candidates = candidates.reindex(columns=new_order)

In [64]:
election_results_raw = pd.read_json("election_datas/SecimSonucIl.json")
election_results_raw.head(10)

,İl Id,İl Adı,Kayıtlı Seçmen Sayısı,Oy Kullanan Seçmen Sayısı,Geçerli Oy Toplamı,RECEP TAYYİP ERDOĞAN,MUHARREM İNCE,KEMAL KILIÇDAROĞLU,SİNAN OĞAN
0,,İLLER TOPLAMI,60.774.566,53.993.683,52.972.934,26.086.102,216.470,23.873.749,2.796.613
1,Oy Oranı,%49.24,%0.4,%45.06,%5.27,,,,
2,1,ADANA,1.615.326,1.414.720,1.388.925,610.132,5.110,706.780,66.903
3,Oy Oranı,%43.92,%0.36,%50.88,%4.81,,,,
4,2,ADIYAMAN,399.303,327.349,319.326,211.389,1.203,99.771,6.963
5,Oy Oranı,%66.19,%0.37,%31.24,%2.18,,,,
6,3,AFYONKARAHİSAR,536.910,484.078,472.801,301.941,2.642,136.507,31.711
7,Oy Oranı,%63.86,%0.55,%28.87,%6.7,,,,
8,4,AĞRI,307.114,223.749,215.851,68.468,873,142.021,4.489
9,Oy Oranı,%31.72,%0.4,%65.79,%2.07,,,,


In [105]:
election_results_for_erdogan = pd.DataFrame()
for i in range(3,election_results_raw.shape[0],2):
    province = election_results_raw.iloc[i-1][1]
    percent = election_results_raw.iloc[i][1]
    percent_float = float(percent.strip("%"))
    new_row = pd.DataFrame({"Province": [province], "Percent": [percent_float]})
    election_results_for_erdogan = pd.concat([election_results_for_erdogan, new_row], ignore_index=True)
    
election_results_for_erdogan

    

    

ValueError: could not convert string to float: ' %43.92 '